In [2]:
pip install unstructured openpyxl networkx msoffcrypto-tool

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.document_loaders import UnstructuredExcelLoader

loader = UnstructuredExcelLoader(r"D:\STUDY\Gen AI\Langchain\hrchatbot\employees.xlsx", mode="elements")
docs = loader.load()

print(len(docs))

docs

1


[Document(metadata={'source': 'D:\\STUDY\\Gen AI\\Langchain\\hrchatbot\\employees.xlsx', 'file_directory': 'D:\\STUDY\\Gen AI\\Langchain\\hrchatbot', 'filename': 'employees.xlsx', 'last_modified': '2025-09-03T00:52:32', 'page_name': 'Sheet1', 'page_number': 1, 'text_as_html': '<table><tr><td>id</td><td>name</td><td>skills</td><td>experience_years</td><td>projects</td><td>availability</td></tr><tr><td>1</td><td>Rachel King</td><td>Python, C++</td><td>4</td><td>Website Redesign, Marketing Automation, Mobile App</td><td>unavailable</td></tr><tr><td>2</td><td>Eva Davis</td><td>AWS, Node.js, Django</td><td>6</td><td>Website Redesign, Analytics Dashboard</td><td>available</td></tr><tr><td>3</td><td>Grace Nguyen</td><td>C++, Flask</td><td>6</td><td>Healthcare Dashboard</td><td>available</td></tr><tr><td>4</td><td>Ian Harris</td><td>SQL, Node.js</td><td>7</td><td>Website Redesign, E-commerce Platform</td><td>available</td></tr><tr><td>5</td><td>Eva Young</td><td>Node.js, Django, AWS, Python</t

In [2]:
type(docs)

list

In [5]:
docs[0]

Document(metadata={'source': 'D:\\STUDY\\Gen AI\\Langchain\\hrchatbot\\employees.xlsx', 'file_directory': 'D:\\STUDY\\Gen AI\\Langchain\\hrchatbot', 'filename': 'employees.xlsx', 'last_modified': '2025-09-03T00:52:32', 'page_name': 'Sheet1', 'page_number': 1, 'text_as_html': '<table><tr><td>id</td><td>name</td><td>skills</td><td>experience_years</td><td>projects</td><td>availability</td></tr><tr><td>1</td><td>Rachel King</td><td>Python, C++</td><td>4</td><td>Website Redesign, Marketing Automation, Mobile App</td><td>unavailable</td></tr><tr><td>2</td><td>Eva Davis</td><td>AWS, Node.js, Django</td><td>6</td><td>Website Redesign, Analytics Dashboard</td><td>available</td></tr><tr><td>3</td><td>Grace Nguyen</td><td>C++, Flask</td><td>6</td><td>Healthcare Dashboard</td><td>available</td></tr><tr><td>4</td><td>Ian Harris</td><td>SQL, Node.js</td><td>7</td><td>Website Redesign, E-commerce Platform</td><td>available</td></tr><tr><td>5</td><td>Eva Young</td><td>Node.js, Django, AWS, Python</td

In [6]:
from langchain.text_splitter import CharacterTextSplitter

# Assuming 'docs' is your loaded Document list
text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(docs)
print(f"Number of chunks: {len(chunks)}")


Number of chunks: 1


In [9]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")

In [10]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)


In [16]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
import os
# llm = Ollama(model="gemma3:12b")

from dotenv import load_dotenv
load_dotenv()

## load the Groq API key
groq_api_key=os.environ['GROQ_API_KEY']
llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="llama-3.3-70b-versatile")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",        # "stuff", "map_reduce", or "refine"
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)


In [18]:
query = "I need someone experienced with node.js for a website designing?"
result = qa_chain({"query": query})

print("Answer:", result['result'])
# Optional: see which chunks were retrieved
for doc in result['source_documents']:
    print(doc.page_content)


Answer: Based on the provided context, the following individuals are experienced with Node.js and are available for a website design project:

1. Eva Davis (6 years of experience)
2. Ian Harris (7 years of experience)
3. Eva Young (9 years of experience)
4. Eva Torres (4 years of experience)
5. Bob Nguyen (1 year of experience)
6. Kevin Robinson (2 years of experience)

These individuals have Node.js listed as one of their skills and are marked as "available". However, it's worth noting that experience years may not be the only factor to consider when choosing a candidate, and other skills or project experience may also be relevant to your needs.
id name skills experience_years projects availability 1 Rachel King Python, C++ 4 Website Redesign, Marketing Automation, Mobile App unavailable 2 Eva Davis AWS, Node.js, Django 6 Website Redesign, Analytics Dashboard available 3 Grace Nguyen C++, Flask 6 Healthcare Dashboard available 4 Ian Harris SQL, Node.js 7 Website Redesign, E-commerce P

# Adding Retriver

In [23]:
query="python"
result=vectorstore.similarity_search(query)
result[0].page_content

'id name skills experience_years projects availability 1 Rachel King Python, C++ 4 Website Redesign, Marketing Automation, Mobile App unavailable 2 Eva Davis AWS, Node.js, Django 6 Website Redesign, Analytics Dashboard available 3 Grace Nguyen C++, Flask 6 Healthcare Dashboard available 4 Ian Harris SQL, Node.js 7 Website Redesign, E-commerce Platform available 5 Eva Young Node.js, Django, AWS, Python 9 Website Redesign available 6 Michael Harris Angular, C++, SQL, Node.js 1 Chatbot, Internal Tool unavailable 7 Hannah White C++, Python, Java, Django 8 CRM unavailable 8 Zane Scott Java, Python 5 Inventory System, E-commerce Platform, CRM unavailable 9 Rachel Anderson Node.js, Flask, React, Java 8 Mobile App, Marketing Automation, Website Redesign available 10 Carol Taylor Flask, React, SQL, Angular 7 Analytics Dashboard, CRM, Mobile App unavailable 11 Yara Martin Python, SQL, React, Java 2 Inventory System available 12 Grace Wilson Django, Node.js 7 CRM unavailable 13 Bob Lewis SQL, Nod

In [24]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [25]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [26]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

In [27]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever= vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020DCDF29BE0>, search_kwargs={})

In [28]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""

from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [29]:
response=retrieval_chain.invoke({"input":"I need someone experienced with node.js for a website designing?"})

In [30]:
response

{'input': 'I need someone experienced with node.js for a website designing?',
 'context': [Document(id='0acb56f8-3a3e-4e1c-ab5f-322d987d245a', metadata={'source': 'D:\\STUDY\\Gen AI\\Langchain\\hrchatbot\\employees.xlsx', 'file_directory': 'D:\\STUDY\\Gen AI\\Langchain\\hrchatbot', 'filename': 'employees.xlsx', 'last_modified': '2025-09-03T00:52:32', 'page_name': 'Sheet1', 'page_number': 1, 'text_as_html': '<table><tr><td>id</td><td>name</td><td>skills</td><td>experience_years</td><td>projects</td><td>availability</td></tr><tr><td>1</td><td>Rachel King</td><td>Python, C++</td><td>4</td><td>Website Redesign, Marketing Automation, Mobile App</td><td>unavailable</td></tr><tr><td>2</td><td>Eva Davis</td><td>AWS, Node.js, Django</td><td>6</td><td>Website Redesign, Analytics Dashboard</td><td>available</td></tr><tr><td>3</td><td>Grace Nguyen</td><td>C++, Flask</td><td>6</td><td>Healthcare Dashboard</td><td>available</td></tr><tr><td>4</td><td>Ian Harris</td><td>SQL, Node.js</td><td>7</td><td

In [31]:
response["answer"]

'Based on the provided context, there are several developers with experience in Node.js who could be suitable for a website design project. Here are some of the top candidates:\n\n1. Eva Davis - With 6 years of experience in AWS, Node.js, and Django, Eva is an excellent choice for a website redesign project. She has experience in building scalable and secure web applications using Node.js, which could be useful for a complex website design project.\n2. Grace Nguyen - Grace has 6 years of experience in C++, Flask, and Angular, making her a strong candidate for a healthcare dashboard project. Her experience in these technologies could also make her a good fit for a website redesign project that requires a high level of customization.\n3. Ian Harris - With 7 years of experience in SQL, Node.js, and Django, Ian is a skilled developer who could handle a website design project. His experience in building scalable web applications using Node.js could be particularly useful for a complex websi